In [6]:
import numpy as np
import matplotlib.pyplot as plt
from GS_algorithm_first_iteration import gsw_output
from GS_algorithm2 import *
from SLM_Control import *
from showSLMPreview import showSLMPreview
from dcam_live_capturing import *
from beam_locator import *
import time
import cv2

In [8]:
slm = SLMControler()

### Initialize camera and capture the background intensity 

To capture the background please block the laser beam

In [3]:
dcam_capture = DcamLiveCapturing(iDevice = 0)

### Background intensity
Block the laser before you take the measurement

In [4]:
background = dcam_capture.capture_single_frame()

print("Background captured", background)

-NG: Dcamapi.init() fails with error -2147483130
Background captured None


### First iteration of GSW without feedback

In [ ]:
### Initilization ###
number_of_rows = 2
number_of_columns = 2
# weight = np.ones(number_of_columns * number_of_rows) / (number_of_columns * number_of_rows)
weight = np.array([0.25, 0.25, 0.25, 0.25])
w0 = 1
interval = 200
error_allowed = 0.01 
Dim = np.array([2, 2])
#####################

# size_real = np.array([1920, 1080]) 
# using the HOLOEYE GAEA-2, the resolution is 3840 x 2160
size_real = np.array([3840, 2160])
Overall_Pattern = np.zeros((2160, 3840), dtype=np.float64)

error = 1

size_real = size_real / Dim 
size_real = size_real.astype(int)
temp = np.zeros(number_of_columns * number_of_rows)
temp[:len(weight)] = weight  
weight_shaped = np.reshape(temp, (number_of_columns, number_of_rows))
weight_shaped = np.flipud(weight_shaped)

Pattern1 = np.zeros((2160, 3840), dtype=np.float64)

for part in range(1, 5):
    # since we are not improving, we are only iterating over this 2 times, first time to use the gsw_output function, and second time to use the gs_iteration_modified function
    for iteration in range (0, 2):
        balance = 0
        if iteration == 0:
            [Pattern_part, phi] = gsw_output(size_real, weight_shaped, interval, number_of_rows, number_of_columns, w0, 0)
            Pattern_last = phi
        else:
            [Pattern_part, phi] = gs_iteration_modified(size_real, weight_shaped, interval, Pattern_last, balance, w0)
            Pattern_last = phi

    
    x = int(Dim[0] - 1 - np.mod(part - 1, Dim[0]))
    y = int(np.floor((part - 1) / Dim[0]))  

    start_x = x * size_real[0]
    end_x = (x+1) * size_real[0]
    start_y = y * size_real[1]
    end_y = (y+1) * size_real[1]
 
    Pattern1[start_y : end_y, start_x : end_x] = Pattern_part

# divide each value in Pattern by 2 * pi
Pattern1 = np.mod(Pattern1, 2 * np.pi)

### Run the beam locator to find the locations of all beams

Loading the beam locator class and display the generated pattern on the SLM

In [ ]:
class beam_locator:
    def __init__(self, image, number_of_rows, number_of_cols):
        self.image = image
        self.cursor_locations = [(1000, 100), (1700, 100), (200, 200), (100, 1700)]  # Initialize cursor locations
        self.dragging = False
        self.current_cursor = None
        self.crosshair_length = 40  # Set the length of the crosshair lines
        self.rows = number_of_rows
        self.cols = number_of_cols
        # initialize all the beam corners
        self.total_num_beams = self.rows * self.cols
        self.beam_corners = np.zeros((self.total_num_beams, 2, 2), dtype=float)

    def draw_crosshair(self, img, center):
        # Draw vertical line
        cv2.line(img, (center[0], center[1] - self.crosshair_length // 2),
                 (center[0], center[1] + self.crosshair_length // 2), (0, 255, 0), 2)
        # Draw horizontal line
        cv2.line(img, (center[0] - self.crosshair_length // 2, center[1]),
                 (center[0] + self.crosshair_length // 2, center[1]), (0, 255, 0), 2)

    def display_image_with_crosshairs(self):
        cv2.namedWindow('Image with Crosshairs', cv2.WINDOW_NORMAL)  # Resizable window

        while True:
            img = self.image.copy()

            # Normalize the image to uint8 if it is uint16
            if img.dtype == np.uint16:
                img = cv2.convertScaleAbs(img, alpha=(255.0/65535.0))

            if len(img.shape) == 2:  # Check if the image is grayscale
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

            # Draw crosshairs at cursor locations
            for loc in self.cursor_locations:
                self.draw_crosshair(img, loc)

            cv2.imshow('Image with Crosshairs', img)

            # Set mouse callback function
            cv2.setMouseCallback('Image with Crosshairs', self.mouse_callback)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cv2.destroyAllWindows()

    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Check if cursor is clicked
            for idx, loc in enumerate(self.cursor_locations):
                if abs(loc[0] - x) < self.crosshair_length // 2 and abs(loc[1] - y) < self.crosshair_length // 2:
                    self.dragging = True
                    self.current_cursor = idx
                    break
        elif event == cv2.EVENT_MOUSEMOVE:
            # Update cursor location if dragging
            if self.dragging:
                self.cursor_locations[self.current_cursor] = (x, y)

        elif event == cv2.EVENT_LBUTTONUP:
            # Stop dragging
            self.dragging = False
            self.current_cursor = None

    def get_cursor_locations(self):
        # sort the cursor locations in the following order, top left, top right, bottom left, bottom right
        self.cursor_locations = sorted(self.cursor_locations, key=lambda x: x[0])
        self.cursor_locations = sorted(self.cursor_locations, key=lambda x: x[1])

        return self.cursor_locations

    def calculate_all_beam_locations(self):
        # from the first and third elements of the cursor_locations list, we can calculate the diameter of a beam
        # the first element of the array is the top left of the beam and the third element represents the bottom right corner of the beam
        self.beam_diameter = np.sqrt((self.cursor_locations[2][0] - self.cursor_locations[0][0])**2 + (self.cursor_locations[2][1] - self.cursor_locations[0][1])**2)
        print("Beam Diameter:", self.beam_diameter)

        # from the first and second element of the list, these are the top left corners of the top left most and top right most beams
        # find the horizontal intervals of all beams
        self.horizontal_intervals = (self.cursor_locations[1][0] - self.cursor_locations[0][0]) / (self.cols - 1)
        # now to find the vertical intervals
        self.vertical_intervals = (self.cursor_locations[3][1] - self.cursor_locations[0][1]) / (self.rows - 1)

        self.horizontal_rise = (self.cursor_locations[1][1] - self.cursor_locations[0][1]) / (self.cols - 1)
        self.vertical_shift = (self.cursor_locations[3][0] - self.cursor_locations[0][0]) / (self.rows - 1)

        # now from these horizontal and vertical intervals, we can calculate the top left corners of all 25 beams and create a box using the diameter calculated as well
        # store the corners in a 25 x 4 array, for each beam, store the four corners of the box

        # Iterate over the rows
        for i in range(self.rows):
            # Iterate over the columns
            for j in range(self.cols):
                # Calculate the top left corner of the current box
                top_left = [self.cursor_locations[0][0] + j * self.horizontal_intervals + i * self.vertical_shift, self.cursor_locations[0][1] + i * self.vertical_intervals + j * self.horizontal_rise]
                
                # Calculate the bottom right corner
                bottom_right = [top_left[0] + self.beam_diameter, top_left[1] + self.beam_diameter]
                
                # Store the corners in the array
                self.beam_corners[i * self.rows + j] = [top_left, bottom_right]

slm.display_data(Pattern1)

Capturing a single frame, and running the beam locator function. 

Drag the cursors to the top left, bottom right corner of the top left beam; the top left corner of the top right beam; and the top left corner of the bottom left beam. After you have placed the cursors in the appropriate locations, pressure "q" on the keyboard to close the window. 

In [ ]:
captured_image = dcam_capture.capture_single_frame()

# Check if an image was captured
if captured_image is not None:
    # # Create a resizable window
    # cv2.namedWindow("Captured Image", cv2.WINDOW_NORMAL)

    # # Display the captured image using OpenCV
    # cv2.imshow("Captured Image", captured_image)
    
    # while True:
    #     if cv2.waitKey(1) & 0xFF == ord('q') or cv2.getWindowProperty("Captured Image", cv2.WND_PROP_VISIBLE) < 1:
    #         break
    # cv2.destroyAllWindows()  # Close the window

    locator = beam_locator(captured_image, number_of_rows, number_of_columns)

    # Display image with crosshairs and allow user interaction
    locator.display_image_with_crosshairs()

    # Get cursor locations
    cursor_locations = locator.get_cursor_locations()

    locator.calculate_all_beam_locations()

    beam_corners = locator.beam_corners

    # store them in a file
    with open("beam_corners.txt", "w") as f:
        for loc in beam_corners:
            top_left, bottom_right = loc
            f.write(f"{top_left[0]}, {top_left[1]}, {bottom_right[0]}, {bottom_right[1]}\n")
else:
    print("No image captured.")

To double check that all the beam corners have been calculated. these values can also be read in the beam_corner.txt file in the directory

In [ ]:
# for debugging purposes
print("Beam Corners:", beam_corners)

### Function to measure the intensity of each beam

In [ ]:
image = captured_image - background

def measure_greyscale_intensity(image, beam_corners):
    """
    Measures the grayscale intensity inside the rectangles described by the coordinates in beam_corners.

    Parameters:
    image (numpy.ndarray): The input image.
    beam_corners (list): A list of tuples, where each tuple contains two pairs of coordinates
                         (top-left and bottom-right) of a beam.

    Returns:
    intensities (list): A list of average grayscale intensities for each beam.
    """
    intensities = []

    for top_left, bottom_right in beam_corners:
        x1, y1 = map(int, top_left)
        x2, y2 = map(int, bottom_right)
        roi = image[y1:y2, x1:x2]
        average_intensity = np.mean(roi)
        intensities.append(average_intensity)

    return intensities

# for testing purposes
# Measure the grayscale intensity inside the rectangles described by the beam corners
intensities = measure_greyscale_intensity(image, beam_corners)

print("Grayscale Intensities for Each Beam:")
for idx, intensity in enumerate(intensities):
    print(f"Beam {idx + 1}: {intensity}")

### Function used to place the generated phase pattern in respective location in the overall phase pattern

In [ ]:
def pattern_placement(part, Pattern, Pattern_part):
    x = int(Dim[0] - 1 - np.mod(part - 1, Dim[0]))
    y = int(np.floor((part - 1) / Dim[0]))  

    start_x = x * size_real[0]
    end_x = (x+1) * size_real[0]
    start_y = y * size_real[1]
    end_y = (y+1) * size_real[1]
 
    Pattern[start_y : end_y, start_x : end_x] = Pattern_part
    return Pattern

In [ ]:
part = 1

[Pattern_part, phi] = gsw_output(size_real, weight_shaped, interval, number_of_rows, number_of_columns, w0, 0)
Pattern_last = phi

Pattern = np.zeros((2160, 3840), dtype=np.float64)

# placing the pattern at the correct location on the whole SLM phase pattern
Pattern = pattern_placement(part, Pattern, Pattern_part)   
Pattern = np.mod(Pattern, 2 * np.pi)

# now we want to display the pattern on the SLM and capture an image of it
slm.display_data(Pattern)

# time delay of 1 seconds 
time.sleep(1)

# capture an image of the SLM
captured_image = dcam_capture.capture_single_frame()

# subtract the background image from the captured image
captured_image = captured_image - background

# calculate the intensity of each beam using the locations found and stored in a file
measured_intensities = measure_greyscale_intensity(captured_image, beam_corners)

In [ ]:
print("Average Grayscale Intensities for Each Beam:")
for idx, intensity in enumerate(measured_intensities):
    print(f"Beam {idx + 1}: {intensity}")

### Rescale the beam intensities

In [ ]:
rescale = 2e-4

In [ ]:
# scale down the measured intensities
intensities = [intensity * rescale for intensity in measured_intensities]

# calculate the std error
std_error = np.std(np.abs(weight - intensities))
print("std Error:", std_error)

# calculate the mean error
mean_error = np.mean(np.abs(weight - intensities))
print("mean Error:", mean_error)

### Running the GSW algorithm

In [ ]:
captured_image = dcam_capture.capture_single_frame()

In [ ]:
from GS_algorithm2 import *

# for part in range(1, 145):
for part in range(1,5):

    # for each part, we are only concerned with that specific part and the rest of the phase pattern would be zero
    # so we first initialize a phase pattern with all zeros
    Pattern = np.zeros((2160, 3840), dtype=np.float64)

    # maximum of 80 iterations
    for iteration in range (0, 81):

        if error < error_allowed:
            break

        if iteration == 0:
            e = 0
            [Pattern_part, phi] = gsw_output(size_real, weight_shaped, interval, number_of_rows, number_of_columns, w0, e)
            Pattern_last = phi

            # placing the pattern at the correct location on the whole SLM phase pattern
            Pattern = pattern_placement(part, Pattern, Pattern_part)   
            Pattern = np.mod(Pattern, 2 * np.pi)

            # now we want to display the pattern on the SLM and capture an image of it
            slm.display_data(Pattern)

            # time delay of 1 seconds 
            time.sleep(1)

            # capture an image of the SLM
            captured_image = dcam_capture.capture_single_frame()
            
            # subtract the background image from the captured image
            captured_image = captured_image - background
            

            # calculate the intensity of each beam using the locations found and stored in a file
            intensities = measure_greyscale_intensity(captured_image, beam_corners)
            # scale down the measured intensities
            intensities = [intensity * rescale for intensity in intensities]

            # print("Average Grayscale Intensities for Each Beam:", intensities)

            # calculate the std error
            std_error = np.std(np.abs(weight - intensities))
            print("std Error:", std_error)

            # calculate the mean error
            mean_error = np.mean(np.abs(weight - intensities))
            print("Mean Error:", mean_error)

        else:
            [Pattern_part, phi] = gsw_output(size_real, weight_shaped, interval, number_of_rows, number_of_columns, w0, mean_error)
            Pattern_last = phi

            Pattern = pattern_placement(part, Pattern, Pattern_part)

            Pattern = np.mod(Pattern, 2 * np.pi)

            # now we want to display the pattern on the SLM and capture an image of it
            slm.display_data(Pattern)

            # time delay of 1 seconds 
            time.sleep(1)

            # capture an image of the SLM
            captured_image = dcam_capture.capture_single_frame()

            time.sleep(1)
            
            # subtract the background image from the captured image
            captured_image = captured_image - background

            # calculate the intensity of each beam using the locations found and stored in a file
            intensities = measure_greyscale_intensity(captured_image, beam_corners)

            # scale down the measured intensities
            intensities = [intensity * rescale for intensity in intensities]

            # print("Average Grayscale Intensities for Each Beam:", intensities)

            # calculate the std error
            std_error = np.std(np.abs(weight - intensities))
            print("std Error:", std_error)

            # calculate the mean error
            mean_error = np.mean(np.abs(weight - intensities))
            print("Mean Error:", mean_error)


In [ ]:
captured_image = dcam_capture.capture_single_frame()

time.sleep(1)

# subtract the background image from the captured image
captured_image = captured_image - background

# calculate the intensity of each beam using the locations found and stored in a file
intensities = measure_greyscale_intensity(captured_image, beam_corners)

# scale down the measured intensities
intensities = [intensity * rescale for intensity in measured_intensities]

# calculate the std error
std_error = np.std(np.abs(weight - intensities))
print("std Error:", std_error)

# calculate the mean error
mean_error = np.mean(np.abs(weight - intensities))
print("mean Error:", mean_error)

In [ ]:
captured_image = dcam_capture.capture_live_images()